In [35]:
import json

import httpx
from folium import Map, TileLayer, LayerControl

In [25]:
titiler_endpoint = "http://127.0.0.1:8000"  # Developmentseed Demo endpoint. Please be kind.
fs_url = "/home/bielstela/vizzuality/dev/foodscapes_tiler/cogs/Foodscapes_combinedGEOTIFF_final_COG.tif"
risk_url = "/home/bielstela/vizzuality/dev/foodscapes_tiler/cogs/biodiversityrisk_COG.tif"

In [13]:
# Fetch File Metadata to get min/max rescaling values (because the file is stored as float32)
r = httpx.get(
    f"{titiler_endpoint}/cog/info",
    params = {
        "url": url,
    }
).json()

bounds = r["bounds"]
print(r)

{'bounds': [-180.0, -90.0, 180.0, 90.0], 'minzoom': 0, 'maxzoom': 1, 'band_metadata': [['1', {}]], 'band_descriptions': [['1', '']], 'dtype': 'int16', 'nodata_type': 'Nodata', 'colorinterp': ['gray'], 'width': 7200, 'count': 1, 'nodata_value': -9999.0, 'driver': 'GTiff', 'overviews': [2, 4, 8], 'height': 3600}


In [14]:
# Fetch File Metadata to get min/max rescaling values (because the file is stored as float32)
r = httpx.get(
    f"{titiler_endpoint}/cog/statistics",
    params = {
        "url": url,
    }
).json()

print(json.dumps(r, indent=4))

{
    "1": {
        "min": 1.0,
        "max": 1110.0,
        "mean": 263.6968045882835,
        "count": 170870.0,
        "sum": 45057873.0,
        "std": 336.2198516659466,
        "median": 102.0,
        "majority": 1.0,
        "minority": 610.0,
        "unique": 86.0,
        "histogram": [
            [
                101152.0,
                7368.0,
                703.0,
                16221.0,
                2924.0,
                14255.0,
                10542.0,
                250.0,
                8238.0,
                9217.0
            ],
            [
                1.0,
                111.9,
                222.8,
                333.70000000000005,
                444.6,
                555.5,
                666.4000000000001,
                777.3000000000001,
                888.2,
                999.1,
                1110.0
            ]
        ],
        "valid_percent": 32.59,
        "masked_pixels": 353418.0,
        "valid_pixels": 170870.0

In [19]:
with open("foodscapes_cmap.json") as f:
    foodscpaes_cmap = json.load(f)

In [38]:
def make_expression(filter_by: list=None):
    if filter_by:
        filter_parts = "|".join([f"(b1=={x})" for x in filter_by])
        return f"where({filter_parts}, b1, 0);"
    else:
        return ""

filter_by = list(range(701, 710))

r1 = httpx.get(
    f"{titiler_endpoint}/cog/tilejson.json",
    params = {
        "url": fs_url,
        "colormap":json.dumps(cm),
        "expression": make_expression(filter_by)
    }
).json()

r2 = httpx.get(
    f"{titiler_endpoint}/cog/tilejson.json",
    params = {
        "url": risk_url,
        "colormap_name": "viridis",
        "expression": "where(b1>=0, b1, 0);"
    }
).json()

m = Map(width='60%', height='60%', zoom_start=2)

fs_layer = TileLayer(
    tiles=r1["tiles"][0],
    opacity=1,
    name="FS",
    attr="Something"
)
fs_layer.add_to(m)

risk_layer = TileLayer(
    tiles=r2["tiles"][0],
    opacity=1,
    name="Risk",
    attr="DigitalGlobe OpenData"
)
risk_layer.add_to(m)

LayerControl().add_to(m)
m

In [30]:
expression

'where((b1==701)|(b1==702)|(b1==703)|(b1==704)|(b1==705)|(b1==706)|(b1==707)|(b1==708)|(b1==709), b1, 0);'